In [8]:
import numpy as np
import pandas as pd
from const import embedding_path
embeddings = np.load(embedding_path, allow_pickle=True)
df = pd.DataFrame(embeddings, columns=['employee', 'embedding'])
print(df.shape)
print(df)

(19, 2)
    employee                                          embedding
0        huy  [0.22873210906982422, -0.9564796686172485, -2....
1        huy  [-0.1191978007555008, -1.648725986480713, -2.4...
2        huy  [0.5203070640563965, -0.9194456338882446, -2.1...
3        huy  [0.20127424597740173, -0.8754792213439941, -2....
4        huy  [-0.11226420104503632, -0.8447368144989014, -2...
5    dung ai  [0.9970278739929199, -1.1126073598861694, -3.1...
6    dung ai  [0.8800203800201416, -1.1757560968399048, -3.0...
7    dung ai  [1.2486956119537354, -1.0430350303649902, -3.2...
8    dung ai  [1.0159077644348145, -1.186078429222107, -2.83...
9    dung ai  [0.7063747644424438, -1.053912878036499, -2.80...
10  dung iot  [-0.014605514705181122, -0.7461525201797485, -...
11  dung iot  [0.022123202681541443, -0.9281516075134277, -0...
12  dung iot  [-0.35685089230537415, -0.5286805629730225, -1...
13  dung iot  [-0.33963507413864136, -0.3695027530193329, -0...
14  dung iot  [-0.53595376014709

In [7]:
# check file in snapshots exist
import os
from pathlib import Path
from const import snap_path

def get_img():
   list_path = []
   for root, dirs, files in os.walk(snap_path, topdown=False):
      for name in files:
         path = "./" + os.path.join(root, name)
         list_path.append(path)
   return list_path
   
print(get_img())

['./static/snap_shots/snap0.jpg', './static/snap_shots/snap7.jpg', './static/snap_shots/snap2.jpg', './static/snap_shots/snap1.jpg', './static/snap_shots/snap4.jpg', './static/snap_shots/snap6.jpg', './static/snap_shots/snap8.jpg', './static/snap_shots/snap3.jpg', './static/snap_shots/snap5.jpg']


In [7]:
try:
    data_form = """
        {
            "action": [{
                "alias": {},
                "condition": [],
                "id": {},
                "trigger": [{ 
                    "entity_id": {},
                    "platform": "state",
                    "from": "off",
                    "to": "on"
                }]
            }]
        }
    """
except Exception as err:
    print(err)

print(data_form.replace("\n", "").format(1, 2, 3))

KeyError: '            "action"'